Data Extraction

In [81]:
import pandas as pd

In [82]:
name_df = pd.read_csv("name_final_data.csv",sep="\t")

In [83]:
name_df.dtypes

TEXT     object
LABEL    object
dtype: object

In [84]:
profane_df = pd.read_csv("profane_words_final.csv",sep="\t")

In [85]:
profane_df.shape

(473002, 2)

In [86]:
profane_df.dtypes

TEXT     object
LABEL    object
dtype: object

In [87]:
final_df = name_df.append(profane_df, ignore_index=True)

In [88]:
final_df=final_df.sample(frac=1).reset_index(drop=True)

In [89]:
final_df.dtypes

TEXT     object
LABEL    object
dtype: object

In [90]:
final_df.isna().sum()

TEXT     0
LABEL    0
dtype: int64

In [91]:
final_df.head(n=10)

,TEXT,LABEL
0,Nar Kumar Updahyay,NAME
1,ahh,NOTNAME
2,raechal akshyata tbh noooo,NOTNAME
3,Bhargaw Mall,NAME
4,suni pani moj,NOTNAME
5,Siya Bhugai,NAME
6,sanji leh stdy gardae,NOTNAME
7,Bibekananda Bucha,NAME
8,Suzeeta Gahatraj,NAME
9,mc salaa chikna,NOTNAME


In [92]:
for l in final_df["TEXT"]:
    if type(l) != str:
        print("bad")
        print(l)

In [93]:
final_df.loc[:,"TEXT"] = final_df.TEXT.apply(lambda x : str(x))

In [94]:
final_df["LABEL"] = final_df["LABEL"].replace(["NAME" , "NOTNAME"] , [1 , 0])

final_df.loc[:,"TEXT"] = final_df.TEXT.apply(lambda x : str.lower(x))

Data splitting

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df["TEXT"], final_df["LABEL"], 
                                                    test_size=.33, random_state=17)

In [96]:
y_test

877561    1
819308    1
567766    0
740179    1
43749     0
162050    1
379742    0
713105    0
697928    0
371805    0
547399    0
123972    0
143809    0
765796    0
448127    0
736745    0
866382    0
730434    1
618429    0
105820    0
501147    0
931301    1
866544    0
481267    1
468390    0
271333    1
834343    0
310631    1
140804    1
326154    1
         ..
9874      1
828177    1
908281    0
459339    0
88437     0
101963    1
896114    0
364491    1
848877    0
302453    1
368790    1
771089    1
598498    1
740537    1
567783    1
684787    0
410895    0
300356    1
103106    1
858557    1
68598     0
242461    1
433302    1
26181     1
183813    0
803007    0
568021    1
215048    1
374774    1
178166    0
Name: LABEL, Length: 311322, dtype: int64

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 20000
tokenizer = Tokenizer(char_level=False, oov_token='UNK',num_words = vocabulary_size)
tokenizer.fit_on_texts(final_df["TEXT"])
sequences = tokenizer.texts_to_sequences(final_df['TEXT'])
data = pad_sequences(sequences,maxlen = 10)

Using TensorFlow backend.


In [16]:
tokenizer.index_word

{1: 'UNK',
 2: 'lol',
 3: 'rana',
 4: 'chandra',
 5: 'kumar',
 6: 'nath',
 7: 'ballav',
 8: 'prasad',
 9: 'bahadur',
 10: 'jung',
 11: 'aditya',
 12: 'september',
 13: 'com',
 14: 'tyo',
 15: 'haha',
 16: 'nai',
 17: 'october',
 18: 'bholi',
 19: 'hai',
 20: 'haru',
 21: 'malai',
 22: 'haina',
 23: 'hola',
 24: 'pani',
 25: 'kati',
 26: 'xaina',
 27: 'tero',
 28: 'lala',
 29: 'timi',
 30: 'bhayo',
 31: 'gardai',
 32: 'chha',
 33: 'okay',
 34: 'aja',
 35: 'maile',
 36: 'bhane',
 37: 'hunxa',
 38: 'shubham',
 39: 'bata',
 40: 'kasto',
 41: 'aah',
 42: 'bhan',
 43: 'na',
 44: 'jana',
 45: 'colz',
 46: 'chaina',
 47: 'pratik',
 48: 'ghar',
 49: 'garna',
 50: 'timro',
 51: 'garne',
 52: 'andresh',
 53: 'tei',
 54: 'huncha',
 55: 'kaile',
 56: 'samma',
 57: 'bhaneko',
 58: 'co',
 59: 'ahh',
 60: 'omar',
 61: 'www',
 62: 'wala',
 63: 'jasto',
 64: 'baje',
 65: 'dherai',
 66: 'kei',
 67: 'hahaha',
 68: 'bhai',
 69: 'kun',
 70: 'lo',
 71: 'sahil',
 72: 'sha',
 73: 'chhaina',
 74: 'bhaye',
 75: 

In [17]:
final_df["LABEL"] = final_df["LABEL"].replace(["NAME" , "NOTNAME"] , [1 , 0])

In [18]:
labels = final_df["LABEL"]

In [19]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense,  LSTM, Conv1D, MaxPooling1D, Dropout, Activation
import numpy as np
from keras.layers import BatchNormalization

In [20]:
model2 = Sequential()
model2.add(Embedding(20000, 50, input_length=10))
model2.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(Dense(50, activation="tanh"))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(Dense(50, activation="tanh"))
model2.add(BatchNormalization())
model2.add(Dropout(0.4))
model2.add(Dense(1, activation="sigmoid"))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(data, np.array(labels), validation_split=0.4, epochs=8)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 566039 samples, validate on 377360 samples
Epoch 1/8
566039/566039 [==============================] - 310s 547us/step - loss: 0.0613 - acc: 0.9788 - val_loss: 0.0125 - val_acc: 0.9966
Epoch 2/8
566039/566039 [==============================] - 225s 398us/step - loss: 0.0122 - acc: 0.9967 - val_loss: 0.0057 - val_acc: 0.9982
Epoch 3/8
566039/566039 [==============================] - 225s 397us/step - loss: 0.0065 - acc: 0.9983 - val_loss: 0.0073 - val_acc: 0.9982
Epoch 4/8
566039/566039 [==============================] - 295s 522us/step - loss: 0.0044 - acc: 0.9990 - val_loss: 0.0045 - val_acc: 0.9991
Epoch 5/8
566039/566039 [==============================] - 300s 531us/step - loss: 0.0030 - acc: 0.9993 - val_loss: 0.0037 - val_acc: 0.9993
Epoch 6/

In [45]:
test_df = pd.read_csv("test_data.csv",sep="\t")

In [46]:
test_df.head(n=5)

,text,label
0,Rajendra Sapkota,1.0
1,Subodh Chandra Shakya,1.0
2,Saugat Adhikari,1.0
3,Sabin Acharya,1.0
4,Binod Chaudhary,1.0


In [47]:
test_df.dtypes

text      object
label    float64
dtype: object

In [48]:
test = test_df["text"]
test_label = test_df["label"]

In [49]:
sequences_test = tokenizer.texts_to_sequences(test)
test_data = pad_sequences(sequences_test,maxlen = 10)

In [50]:
test_pred = model2.predict(test_data)

In [51]:
new_pred = []
for i in test_pred:
    if i>0.5:
        new_pred.append("NAME")
    else:
        new_pred.append("NOTNAME")

In [52]:
result = [(i,j)for i,j in zip(test,new_pred)]

In [53]:
result

[('Rajendra Sapkota', 'NAME'),
 ('Subodh Chandra Shakya', 'NAME'),
 ('Saugat Adhikari', 'NAME'),
 ('Sabin Acharya', 'NAME'),
 ('Binod Chaudhary', 'NAME'),
 ('Keshav Kumar Aryal', 'NAME'),
 ('Jeet Bahadur Khamcha', 'NAME'),
 ('Jit Khamcha', 'NAME'),
 ('Ishwor Subedi', 'NAME'),
 ('Nitesh Ghimire', 'NAME'),
 ('Asmita Khakurel', 'NAME'),
 ('Asmi Bhujhel', 'NOTNAME'),
 ('Abin Khatiwada', 'NAME'),
 ('Binod Pudasaini', 'NAME'),
 ('Hari Lama', 'NAME'),
 ('Ram Chandra Lamichhanne', 'NAME'),
 ('Rajnita randi', 'NOTNAME'),
 ('Sambriddhi Sharma', 'NAME'),
 ('Archana Shakya', 'NAME'),
 ('mula', 'NOTNAME'),
 ('tero bauuu', 'NOTNAME'),
 ('aalu jasto', 'NOTNAME'),
 ('chor', 'NOTNAME'),
 ('bhaate', 'NOTNAME'),
 ('khaate', 'NOTNAME'),
 ('ganjeee', 'NOTNAME'),
 ('maam paka', 'NOTNAME'),
 ('railaa', 'NOTNAME'),
 ('raila', 'NOTNAME'),
 ('kera jasto ', 'NOTNAME'),
 ('tero bau', 'NOTNAME'),
 ('badar jasto tah', 'NOTNAME'),
 ('baadar', 'NOTNAME'),
 ('monkey', 'NOTNAME'),
 ('donkey', 'NOTNAME'),
 ('tah gadha',

On testing data


In [98]:
X_test.shape,y_test.shape

((311322,), (311322,))

In [99]:
sequences_for_test = tokenizer.texts_to_sequences(X_test)
final_test_data = pad_sequences(sequences_for_test,maxlen = 10)

In [100]:
y_test

877561    1
819308    1
567766    0
740179    1
43749     0
162050    1
379742    0
713105    0
697928    0
371805    0
547399    0
123972    0
143809    0
765796    0
448127    0
736745    0
866382    0
730434    1
618429    0
105820    0
501147    0
931301    1
866544    0
481267    1
468390    0
271333    1
834343    0
310631    1
140804    1
326154    1
         ..
9874      1
828177    1
908281    0
459339    0
88437     0
101963    1
896114    0
364491    1
848877    0
302453    1
368790    1
771089    1
598498    1
740537    1
567783    1
684787    0
410895    0
300356    1
103106    1
858557    1
68598     0
242461    1
433302    1
26181     1
183813    0
803007    0
568021    1
215048    1
374774    1
178166    0
Name: LABEL, Length: 311322, dtype: int64

In [101]:
# evaluate the model
scores = model2.evaluate(final_test_data, y_test, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))

acc: 99.94%


In [102]:
# serialize model to JSON
model_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model2.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [104]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [105]:
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(final_test_data, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 99.94%
